In [2]:
from langchain.agents import initialize_agent,Tool,AgentType
from langchain.llms import OpenAI
import os
import dotenv

dotenv.load_dotenv()

llm = OpenAI(temperature=0,api_key=os.getenv("openaiKey"))

def resposta_simples(query):
    return f"a resposta para '{query}' e paris"

tools = [
    Tool(
        name="resposta simples",
        func=resposta_simples,
        description="uma ferramenta simples para reponder perguntas diretas"

    )
]


agent = initialize_agent(tools,llm,agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,verbose=True)


input_query = "qual a capital da frança?"

response = agent.run(input_query)

print(response)




> Entering new AgentExecutor chain...
 I should use a simple tool to answer this question
Action: resposta simples
Action Input: "qual a capital da frança?"
Observation: a resposta para 'qual a capital da frança?' e paris
Thought: I now know the final answer
Final Answer: Paris

> Finished chain.
Paris


In [18]:
from langchain.agents import AgentExecutor
import sqlite3

In [19]:
def create_database():
    conn = sqlite3.connect(":memory:")
    c = conn.cursor()
    c.execute("""CREATE TABLE users (id INTEGER PRIMARY KEY, name TEXT, age INTEGER)""")
    c.execute("INSERT INTO users (name, age) VALUES ('João', 25), ('Maria', 30)")
    conn.commit()
    return conn

def search_user_db(user_name):
    conn = create_database()
    c = conn.cursor()
    c.execute("SELECT name, age FROM users WHERE name = ?", (user_name, ))
    result = c.fetchone()
    conn.close()
    if result:
        return f"{result[0]} tem {result[1]} anos"
    else: 
        return "Usuário não encontrado"
    
def sum_numbers(numbers):
    if isinstance(numbers, str):
        numbers = eval(numbers)
    
    if isinstance(numbers, list):
        total = sum(numbers)
        return f"A soma dos números {numbers} é {total}"
    else:
        return "Erro: A entrada não é uma lista de números válida."

In [20]:
tools = [
    Tool(
        name="Buscar Usuário no BD",
        func=search_user_db,
        description="Procura informações sobre um usuário na base de dados"
    ),
    Tool(
        name="Calcular Soma",
        func=sum_numbers,
        description="Calcula a soma de uma lista de números"
    )
]

In [21]:
agent = initialize_agent(tools, llm, agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

input_query="Qual é a soma de 5, 10 e 15? Quantos anos Maria tem?"
response = agent.run(input_query)
print(response)



> Entering new AgentExecutor chain...
 I need to calculate the sum of 5, 10, and 15 and find out Maria's age.
Action: Calcular Soma
Action Input: 5, 10, 15
Observation: Erro: A entrada não é uma lista de números válida.
Thought: I need to find out Maria's age.
Action: Buscar Usuário no BD
Action Input: Maria
Observation: Maria tem 30 anos
Thought: I now know the final answer.
Final Answer: A soma de 5, 10 e 15 é 30 e Maria tem 30 anos.

> Finished chain.
A soma de 5, 10 e 15 é 30 e Maria tem 30 anos.
